*A document in progress regarding my analyses of COBALT output (1/2)* 

### Install Climate Data Operators (CDO)
```
# Install CDO with grib2, netcdf, HDF5 support
# Adapted from https://gist.github.com/mainvoid007/e5f1c82f50eb0459a55dfc4a0953a08e

mkdir /opt/cdo-install
sudo -s -H
home=/opt/cdo-install
apt-get update
apt-get install -y wget build-essential checkinstall unzip m4 curl libcurl4-gnutls-dev

# download, compile and install --> zlib
cd $home
wget ftp://ftp.unidata.ucar.edu/pub/netcdf/netcdf-4/zlib-1.2.8.tar.gz
tar -xzvf zlib-1.2.8.tar.gz
cd zlib-1.2.8
./configure -prefix=/opt/cdo-install
make && make check && make install

# download, compile and install --> hdf5
cd $home
wget ftp://ftp.unidata.ucar.edu/pub/netcdf/netcdf-4/hdf5-1.8.13.tar.gz
tar -xzvf hdf5-1.8.13.tar.gz
cd hdf5-1.8.13
./configure -with-zlib=/opt/cdo-install -prefix=/opt/cdo-install CFLAGS=-fPIC
make && make check && make install

# download, compile and install --> netCDF
cd $home
wget ftp://ftp.unidata.ucar.edu/pub/netcdf/netcdf-4.5.0.tar.gz
tar -xzvf netcdf-4.5.0.tar.gz 
cd netcdf-4.5.0/
CPPFLAGS=-I/opt/cdo-install/include LDFLAGS=-L/opt/cdo-install/lib ./configure -prefix=/opt/cdo-install CFLAGS=-fPIC
make && make check && make install

# download, compile and install --> jasper
cd $home
wget http://www.ece.uvic.ca/~frodo/jasper/software/jasper-1.900.1.zip
unzip jasper-1.900.1.zip
cd jasper-1.900.1
./configure -prefix=/opt/cdo-install CFLAGS=-fPIC
make && make check && make install

# download, compile and install --> grib_api
cd $home
wget https://software.ecmwf.int/wiki/download/attachments/3473437/grib_api-1.24.0-Source.tar.gz?api=v2 -O grib_api-1.24.0.tar.gz
tar -xzvf grib_api-1.24.0.tar.gz
cd grib_api-1.24.0-Source
./configure -prefix=/opt/cdo-install CFLAGS=-fPIC -with-netcdf=/opt/cdo-install -with-jasper=/opt/cdo-install
make && make check && make install

# download, compile and install --> cdo
cd $home
wget https://code.mpimet.mpg.de/attachments/download/15653/cdo-1.9.1.tar.gz
tar -xvzf cdo-1.9.1.tar.gz
cd cdo-1.9.1
./configure -prefix=/opt/cdo-install CFLAGS=-fPIC -with-netcdf=/opt/cdo-install -with-jasper=/opt/cdo-install -with-hdf5=/opt/cdo-install -with-grib_api=/opt/cdo-install/grib_api-1.24.0-Source
make && make check && make install
```

### Making clips

First, clone the mkmov repo:
```
git clone https://github.com/chrisb13/mkmov
```
Ensure that ImageMagick is installed:
```
identify -version
```
Command line may suggest that you install it; if so:
```
sudo apt install imagemagick
```

Next, view the variable of choice in ncview and use ImageMagick's convert command to produce a GIF:

```ncview -frames *hioekg_his_04750_05110* &
convert frame.*.png outmovie.gif```


*In progress*
```
### Bash script looping across 10 time series files;
# avgs over each one, then
# avgs over the 10 avgs
# = one 10 month avg

#!/bin/bash

printf -v PATHin "/share/frinkraid/powellb/fleat/inner-nlm-w4d-2015-2016/output$
printf -v PATHout "/share/frinkraid4/szedler/fleat/inner-nlm-w4d-2015-2016/anal$
  
  t0=5479
for ((i=0; i<22; i++ ));
do
k=$((t0))
echo $k
printf -v FNin "%sfleat_inner_his_%05d.nc" $PATHin $k
printf -v FNout "%stemp_%05d.nc" $PATHout $k
echo $FNin
echo $FNout
ncks -d ocean_time,0,239 -v temp $FNin $FNout
t0=$((t0+30))
done
cd $PATHout
ncra temp_0*nc temp_mean.nc
```

*In progress*
import numpy as np
import seapy
depths = np.array([1,10,50,100,125,150,200], dtype=object)
nx = 4*4/111
ny = 4*4/111
seapy.roms.interp.to_zgrid('hiig_cobalt_avg.nc', 'out.nc', depth=depths, threads=4, nx=nx, ny=ny)